### **Load Environment variables from .env file**

In [26]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown, Image
from neo4j import GraphDatabase

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
api_version = "2024-02-01"

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

llm = AzureChatOpenAI(
    model=OPENAI_GPT4_DEPLOYMENT_NAME,
    azure_deployment=OPENAI_GPT4_DEPLOYMENT_NAME,
    api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    openai_api_version=api_version,
)


In [27]:
def call_openAI(text):
    response = llm.chat.completions.create(
        model=OPENAI_GPT4_DEPLOYMENT_NAME,
        messages = text,
        temperature=0.0
    )
    return response.choices[0].message.content

In [28]:
# download the wikipedia pages for "Elizabeth II"
from langchain.document_loaders import WikipediaLoader

raw_documents = WikipediaLoader(query="'Elizabeth II").load()
raw_documents[:3]

c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[Document(metadata={'title': 'Elizabeth II', 'summary': "Elizabeth II (Elizabeth Alexandra Mary; 21 April 1926 – 8 September 2022) was Queen of the United Kingdom and other Commonwealth realms from 6 February 1952 until her death in 2022. She was queen regnant of 32 sovereign states over the course of her lifetime and remained the monarch of 15 realms by the time of her death. Her reign of 70 years and 214 days is the longest of any British monarch or female monarch, and the second-longest verified reign of any monarch of a sovereign state in history.\nElizabeth was born in Mayfair, London, during the reign of her paternal grandfather, King George V. She was the first child of the Duke and Duchess of York (later King George VI and Queen Elizabeth The Queen Mother). Her father acceded to the throne in 1936 upon the abdication of his brother Edward VIII, making the ten-year-old Princess Elizabeth the heir presumptive. She was educated privately at home and began to undertake public dutie

In [47]:
# split the documents into chunks
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=30)
documents = text_splitter.split_documents(raw_documents)

In [48]:
len(documents)

24

In [49]:
# define embeddings 
embeddings = AzureOpenAIEmbeddings(
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    openai_api_version=api_version,
    chunk_size = 1
)

In [50]:
#load the documents into Faiss - an in memory vector database
from langchain.vectorstores import FAISS

db = FAISS.from_documents(documents=documents, embedding=embeddings)
# save the FAISS index to disk
db.save_local("./dbs/documentation/faiss_index")

In [51]:
# load the vector store to memory
from langchain.chains import RetrievalQA

vectorStore = FAISS.load_local("./dbs/documentation/faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # returns 3 most similar vectors/documents
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [52]:
# ask a question
question = "Which house did Elizabeth II belong to?"
r = qa.invoke({"query": question})

display(HTML(r['result']))

In [53]:
# ask a question
question = "Describe the family and relationships of Elizabeth II"
r = qa.invoke({"query": question})

display(HTML(r['result']))

In [54]:
# convert the text documents to graph documents
from langchain_experimental.graph_transformers import LLMGraphTransformer

# documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [40]:
#load the graph documents into Neo4j
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()
graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)

In [42]:
# query the graphDB
query = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

driver = GraphDatabase.driver(uri = NEO4J_URI,auth = (NEO4J_USERNAME,NEO4J_PASSWORD))
session = driver.session()
graph_data = session.run(query).graph()

In [43]:
#visualize the graph
from yfiles_jupyter_graphs import GraphWidget
GraphWidget(graph = graph_data)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [44]:
# add the embeddings to the graph
from langchain_community.vectorstores import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [45]:
query = "Describe the family and relationships of Elizabeth II"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: Elizabeth II (Elizabeth Alexandra Mary; 21 April 1926 – 8 September 2022) was Queen of the United Kingdom and other Commonwealth realms from 6 February 1952 until her death in 2022. She was queen regnant of 32 sovereign states over the course of her lifetime and remained the monarch of 15 realms by the time of her death. Her reign of 70 years and 214 days is the longest of any British monarch or female monarch, and the second-longest verified reign of any monarch of a sovereign state in history.
Elizabeth was born in Mayfair, London, during the reign of her paternal grandfather, King George V. She was the first child of the Duke and Duchess of York (later King George VI and Queen Elizabeth The Queen Mother). Her father acceded to the throne in 1936 upon the abdication of his brother Edward VIII, making the ten-year-old Princess Elizabeth the heir presumptive. She was educated privately at home and began to undertake public duties during the Second World War, serving in the Auxil